In [5]:
import pandas as pd
from swarm_sim import *
from metrics import *
import random 
import nbimporter
from metrics import path_disparity, path_redundancy, routing_cost, flow_robustness, network_efficiency, critical_nodes
from tqdm import tqdm

### extraction Data 

In [6]:
df = pd.read_csv("Traces.csv", header=None)  

# Créer les noms de colonnes pour le temps
time_cols = [str(i+1) for i in range(10000)]
df.columns = time_cols

#générer les noms des lignes (cordonnées des satellites):
coord_names = []
for i in range(1, 101):  # 100 satellites
    coord_names.append(f"satx{i}")
    coord_names.append(f"saty{i}")
    coord_names.append(f"satz{i}")
df["coords"] = coord_names
df = df.set_index("coords")

dft = df.transpose()  # maintenant chaque ligne = un instant

### convert to swarm

In [7]:
connection_range = 60000
portee1 =20000
portee2=40000
portee3= 60000
nanosat_count = 40
max_temps = 100  # pour test rapide
t_attack = 50 
resultats=[]

# Liste des satellites encore actifs
satellites_actifs = set(range(1, nanosat_count + 1))

for t in tqdm(range(max_temps), desc="Simulation en cours"):
    print(f"\n--- Temps t = {t} ---")
    coords = dft.iloc[t]
    nodes = []
    
    # Créer les objets Node
    for i in satellites_actifs:
        x = coords[f'satx{i}']
        y = coords[f'saty{i}']
        z = coords[f'satz{i}']
        nodes.append(Node(id=i, x=x, y=y, z=z))

    swarm = Swarm(connection_range=connection_range, nodes=nodes)

    # Créer un graphe pondéré
    G = nx.Graph()
    for node in nodes:
        G.add_node(node.id)

    # Ajouter les arêtes avec poids selon la distance
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            dist = nodes[i].compute_dist(nodes[j])
            if dist <= portee1:
                weight = 1
            elif dist <= portee2:
                weight = 2
            elif dist <= portee3:
                weight = 3
            else:
                continue  
            G.add_edge(nodes[i].id, nodes[j].id, weight=weight)
        # ----- Étape 1 : Métriques de base (graphe non divisé) -----
    metriques_avant = {
        "time": t,
        "algo": "none",
        "attack": "none",
        "efficacite":0, # network_efficiency(G),
        "cout": 0, #routing_cost(G),
        "robustesse": 0, #flow_robustness(G),
        "redondance": path_redundancy(G),
        "disparite": 0, #path_disparity(G)
        "criticite": 0, #critical_nodes(G)
    }
    resultats.append(metriques_avant)
    print("  > Métriques avant division calculées.")
    # ----- Étape 2 : Pour chaque algorithme de division -----
   
    groupes = swarm.RND(n=10)
    algo = 'RND'
    # Construire group_dict
    group_dict = {}
    for group_id, sub_swarm in groupes.items():
        for n in sub_swarm.nodes:
            group_dict[n.id] = group_id

    # ----- Métriques post-division -----
    metriques_div = {
        "time": t,
        "algo": algo,
        "attack": "none",
        "efficacite": network_efficiency(G, divided=True, group_dict=group_dict),
        "cout": routing_cost(G, divided=True, group_dict=group_dict),
        "robustesse": flow_robustness(G, divided=True, group_dict=group_dict),
        "redondance": path_redundancy(G, divided=True, group_dict=group_dict),
        "disparite": 0, #path_disparity(G, divided=True, group_dict=group_dict)
         "criticite": critical_nodes(G, divided=True, group_dict=group_dict)
    }
    
    resultats.append(metriques_div)
    print("  > Métriques après division calculées.")
   
    # ----- Étape 3 : Appliquer les attaques -----
    attack_type = "none"
    nb_noeuds_supprimes = 2
    G_copy = G.copy()
    
    if t == t_attack:
        attack_type = "random"
        if attack_type == "cible":
            BC = node_criticality(G_copy, divided=True, group_dict=group_dict)
        if attack_type == "random":
            nodes_to_remove = random.sample(list(G_copy.nodes), min(nb_noeuds_supprimes, len(G_copy)))
        else:  # attaque ciblée
            nodes_to_remove = sorted(BC, key=BC.get, reverse=True)[:nb_noeuds_supprimes]

        G_copy.remove_nodes_from(nodes_to_remove)

        # Retirer les satellites attaqués de la simulation
        satellites_actifs.difference_update(nodes_to_remove)

        # ----- Métriques post-attaque -----
    metriques_attaque = {
            "time": t,
            "algo": algo,
            "attack": attack_type,
            "efficacite": network_efficiency(G_copy, divided=True, group_dict=group_dict),
            "cout": routing_cost(G_copy, divided=True, group_dict=group_dict),
            "robustesse": flow_robustness(G_copy, divided=True, group_dict=group_dict),
            "redondance": path_redundancy(G_copy, divided=True, group_dict=group_dict),
            "disparite": 0, #path_disparity(G_copy, divided=True, group_dict=group_dict)
            "criticite": critical_nodes(G_copy, divided=True, group_dict=group_dict)
        }
    resultats.append(metriques_attaque)
    print("  > Métriques après attaque calculées.")




Simulation en cours:   0%|                                                                     | 0/100 [00:00<?, ?it/s]


--- Temps t = 0 ---
  > Métriques avant division calculées.
  > Métriques après division calculées.


Simulation en cours:   1%|▌                                                            | 1/100 [00:05<09:52,  5.98s/it]

  > Métriques après attaque calculées.

--- Temps t = 1 ---
  > Métriques avant division calculées.
  > Métriques après division calculées.


Simulation en cours:   2%|█▏                                                           | 2/100 [00:11<09:45,  5.98s/it]

...

  > Métriques après attaque calculées.

--- Temps t = 98 ---
  > Métriques avant division calculées.
  > Métriques après division calculées.


Simulation en cours:  99%|███████████████████████████████████████████████████████████▍| 99/100 [04:54<00:02,  2.20s/it]

  > Métriques après attaque calculées.

--- Temps t = 99 ---
  > Métriques avant division calculées.
  > Métriques après division calculées.


Simulation en cours: 100%|███████████████████████████████████████████████████████████| 100/100 [04:56<00:00,  2.96s/it]

  > Métriques après attaque calculées.


In [8]:
# ========== Sauvegarde en CSV ==========
df_res = pd.DataFrame(resultats)
df_res.to_csv("resultats_swarm_simulation.csv", index=False)
import os 
fichier="resultats_swarm_simulation.csv"
if os.path.exists(fichier):
    print("existe")
else:
    print('existe pas')

existe
